# Domácí úkol:
Pomocí vlastních tříd a dědičnosti vytvořte "soubor tříd" = nástroj pro derivaci funkcí.

## Jak to má fungovat:

- derivace je jednoduše algoritmizovatelná, neboť se řídí prakticky pouze třemi pravidly (podílu se dá vyhnout pomocí násobení funkcí $\frac{1}{f(x)}$):
    - derivace součtu je součet derivací
    - pravidlo pro derivaci součinu
    - derivace funkce je derivovaná funkce násobená derivovaným argumentem

- tedy stačí implementovat tato tři základní pravidla a pro každou elementární funkci (sin, cos, exp, ln, x^n, ...) vědět, jak vypadá její derivace
- celý tento proces lze reprezentovat pomocí tříd a dědičnosti
- trik je v tom, že se výrazy automaticky skládají do stromu (pro nás v podstatě skrytého), a tedy každý výraz je navenek pouze buď sčítání, násobení, nebo elementární funkce (a o jejich argumentech platí totéž)

## Podrobnější zadání:
Konkrétněji:
- Vytvořte třídu **Operator**
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor (zde stačí `pass`)
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
- Vytvořte třídu **BinaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `levy_argument` - levý operand
        - `pravy_argument` - pravý operand
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argumenty
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru
- Vytvořte třídu **UnaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `argument` - argument funkce 
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argument
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru

Pro počítání derivací použijte standardní pravidla:
- derivace součtu je součet derivací
- pravidlo pro derivaci součinu
- derivace funkce je derivovaná funkce násobená derivovaným argumentem

Pro **BinaryOperator** a **UnaryOperator** implementujte také metody implementující standardní operátory:
- `__add__()` - implementuje operátor `+`
- `__sub__()` - implementuje operátor `-`
- `__mul__()` - implementuje operátor `*`

Pro **BinaryOperator** implementujte dvě dceřiné třídy:
- **Addition** - implementuje operátor `+`
- **Multiplication** - implementuje operátor `*`

Pro **UnaryOperator** implementujte tyto dceřiné třídy:
- **exp** - implementuje funkci `exp`
- **sin** - implementuje funkci `sin`
- **cos** - implementuje funkci `cos`
- **ln** - implementuje funkci `ln`
- **frc** - implementuje funkci `1/x`
- **identity** - implementuje identitu `x`
- **cst** - implementuje konstantu `c`
- **mocnina** - implementuje mocninu `x^n`

U všech funkcí implementujte adekvátně metodu `diff()` a `__str__()`, tak aby se funkce vypisovaly v přehledné podobě.

Ozkoušejte na následujících funkcích:
- $sin(x + cos(x*x)) + 1$
- $x^2 + 2x + 1 + 2x^2$
- $x^2 + 2x + exp(x)$

In [32]:
# Base class for binary operations and elementary functions
class Operator:
    def __init__(self):
        pass
    
    def __str__(self):
        return type(self).__name__
    
    def __repr__(self):
        return self.__str__()
    
    def diff(self):
        pass
    
    def __add__(self, other):
        return Addition(self, other)
    
    def __sub__(self, other):
        return Addition(self, Multiplication(cst(-1), other))
    
    def __mul__(self, other):
        return Multiplication(self, other)
    
# Base class for binary operations between el. functions
class BinaryOperator(Operator):
    def __init__(self, levy_argument, pravy_argument):
        self.levy_argument = levy_argument
        self.pravy_argument = pravy_argument
        
# Base class for elementary functions
class UnaryOperator(Operator):
    def __init__(self, argument):
        self.argument = argument
    
    def __str__(self):
        return type(self).__name__ + "(" + str(self.argument) + ")"
        
class Addition(BinaryOperator):
    def __str__(self):
        return str(self.levy_argument) + " + " + str(self.pravy_argument)
    
    def diff(self):
        return self.levy_argument.diff() + self.pravy_argument.diff() # Derivative of a sum: (f + g)' = f' + g'
    
class Multiplication(BinaryOperator):
    def __str__(self):
        return "(" + str(self.levy_argument) + ")*(" + str(self.pravy_argument) + ")"
    
    def diff(self):
        return self.levy_argument.diff() * self.pravy_argument + self.levy_argument * self.pravy_argument.diff() # Derivative of a product: (f * g)' = f' * g + f * g'
    
class cst(UnaryOperator):
    def __init__(self, value):
        self.value = value
    
    def __str__(self):
        return str(self.value)
    
    def diff(self):
        return cst(0) # Derivative of a constant: c' = 0
    
class identity(UnaryOperator):
    def __init__(self):
        pass
    
    def __str__(self):
        return "x"
    
    def diff(self):
        return cst(1) # Derivative of an identical function: x' = 1
    
class mocnina(UnaryOperator):
    def __init__(self, argument, exponent):
        self.argument = argument
        self.exponent = exponent
    
    def __str__(self):
        return "(" + str(self.argument) + ")^" + str(self.exponent)

    def diff(self):
        return cst(self.exponent) * mocnina(self.argument, cst(self.exponent - 1)) * self.argument.diff() # Derivative of a power: (x^n)' = n*(x^(n-1))
    
class sin(UnaryOperator):
    def diff(self):
        return cos(self.argument) * self.argument.diff() # ( sin(f(x)) )' = cos(f(x)) * (f(x))'

class cos(UnaryOperator):
    def diff(self):
        return cst(-1) * sin(self.argument) * self.argument.diff() # ( cos(f(x)) )' = -sin(f(x)) * (f(x))'

class exp(UnaryOperator):
    def diff(self):
        return exp(self.argument) * self.argument.diff()  # ( e^(f(x)) )' = e^(f(x)) * (f(x))'
    
class ln(UnaryOperator):
    def diff(self):
        return frc(self.argument) * self.argument.diff() # ( ln(f(x)) )' = ( 1/(f(x)) ) * (f(x))'

class frc(UnaryOperator):
    def __str__(self):
        return "1/(" + str(self.argument) + ")"
    
    def diff(self):
        return cst(-1) * mocnina(frc(self.argument), 2) * self.argument.diff() # ( 1/(f(x)) )' = ( -1/((f(x))^2) ) * (f(x))'

In [ ]:
# takto by se to mělo používat
x = identity()
# sin(x + cos(x*x)) + 1
f1 = sin(x + cos(x*x)) + cst(1)
print(f1)
print(f1.diff())

můj výstup byl:

`sin(x + cos((x)*(x))) + 1`

`(cos(x + cos((x)*(x))))*(1 + ((-1)*(sin((x)*(x))))*((1)*(x) + (x)*(1))) + 0`

In [ ]:
x = identity()
# x ^ 2 + 2x + 1 + 2x ^ 2
f2 = mocnina(x,2) + cst(2)*x + cst(1) + cst(2)*mocnina(x,2)
print(f2)
print(f2.diff())


můj výstup byl:

`(x)^2 + (2)*(x) + 1 + (2)*((x)^2)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + 0 + (0)*((x)^2) + (2)*(((2)*((x)^1))*(1))`

In [ ]:
x = identity()
# x ^ 2 + 2x + exp(x)
f3 = mocnina(x,2) + cst(2)*x + exp(x)
print(f3)
print(f3.diff())


můj výstup byl:

`(x)^2 + (2)*(x) + exp(x)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + (exp(x))*(1)`


# Bonusový úkol:
Doplňte ke všem třídám metodu `simplify()`, která provede zjednodušení výrazu. Například:
- součet s nulou vrátí druhý operand
- součin s nulou vrátí nulu
- mocnina s exponentem 0 vrátí 1
- mocnina s exponentem 1 vrátí první argument
- součín dvou stejných argumentů vrátí mocninu s exponentem 2
- součin konstant vrátí konstantu se součinem
- ...

Doplňte metody `__add__, __radd__, __mul__, __rmul__, ...`, tak aby podporovaly operace s čísly (např. `2 + x` nebo `x * 3`), a zjednodušovaly zadávání výrazů. 

In [108]:
# BONUS implements:
# - Equivalence of operators (important for simplification processes)
# - Support for typing constants as integers or floats within binary operations (such as: 2 + x, sin(x) - 3, 2 * x)
# - Support for typing powers as python operators: mocnina(a, int) <=> a ** int (such as: 2 ** 5, x ** 7)
# - Simplifications of sums:
#   - a + cst == cst + a             (moves constant always to the left)
#   - cst1 + cst2 == cst3            (calculates a sum of constants)
#   - a + 0 == 0 + a == a            (reduces sums containing zero operands)
#   - n*a + m*a == (n + m)*a         (factoring out)
#   - (cst + a) + b = cst + (a + b)  (takes constants out of nested additions)  ->  (this together with moving constants to left enables grouping and calculating constants in multiple sums)
#     (Does not support grouping of any same UnaryOperators, such as: 2*x + 3 + sin(x) + 5*(x^1) == 3 + 7*x + sin(x))
#     (This is too complicated)
# - Simplified rendering of products  (places no parentheses around factors which does not need it, such as constants or identities; and skips operator* if it is case of cst*identity, such as 5x)
# - Simplifications of products:
#   - a * cst == cst * a             (moves constant always to the left)
#   - cst1 * cst2 == cst3            (calculates a product of constants)
#   - a * 1 == 1 * a == a            (reduces products containing unit-constant operands)
#   - a * 0 == 0 * a == 0            (reduces products containing zero operands)
#   - (a^n) * (a^m) = a^(n + m)      (shortens products of powers having the same base)
#   - (cst * a) * b = cst * (a * b)  (takes constants out of nested multiplications)  ->  (this together with moving constants to left enables grouping and calculating constants in multiple products)
#     (Does not support grouping of any same UnaryOperators, such as: 2 * (x + 3) * sin(x) * 5*(x + 3) == 2 * 6*(x + 3) * sin(x))
#     (This is too complicated)
# - Simplified rendering of powers  (places no parentheses around factors which does not need it, such as constants or identities)
# - Simplifications of powers:
#   - cst1 ^ cst2 == cst3            (calculates a power of constants)
#   - a ^ 1 == a                     (reduces unit-powers)
#   - a ^ 0 == 1                     (reduces zero-powers)
#   - (a^n) ^ m = a^(n * m)          (reduces powers of powers)
# - Identities are still treated as only "x" identity, thus they are converted to string as "x"
#   (but may be implemented so that they distinguish a variable taken in the constructor as string, which would make possible to differentiate with respect one of given variables)
# - Fractions are still treated as a kind of UnaryOperator 
#   (but may be implemented as a kind of BinaryOperator, which would provide a possiblitiy to simplify typing fractions)
#  
# (ALL SIMPLIFICATIONS WORK RECURSIVELY)


# Base class for binary operations and elementary functions
class Operator:
    def __init__(self):
        pass

    def __str__(self):
        return type(self).__name__

    def __repr__(self):
        return self.__str__()

    def diff(self):
        pass

    def __add__(self, other):
        # Supports numbers as left operands of addition
        if isinstance(other, (int, float)):
            other = cst(other)
        return Addition(self, other)
    
    def __radd__(self, other):
        # Supports numbers as right operands of addition
        return self.__add__(other)

    def __sub__(self, other):
        # Supports numbers as left operands of subtraction
        if isinstance(other, (int, float)):
            other = cst(other)
        return Addition(self, Multiplication(cst(-1), other))
    
    def __rsub__(self, other):
        # Supports numbers as right operands of subtraction
        return self.__sub__(other)

    def __mul__(self, other):
        # Supports numbers as left operands of multiplication
        if isinstance(other, (int, float)):
            other = cst(other)
        return Multiplication(self, other)
    
    def __rmul__(self, other):
        # Supports numbers as right operands of multiplication
        return self.__mul__(other)
    
    def __pow__(self, other):
        return mocnina(self, other)
    
    def __eq__(self, other):
        pass
    
    def simplify(self):
        return self


# Base class for binary operations between el. functions
class BinaryOperator(Operator):
    def __init__(self, levy_argument, pravy_argument):
        self.levy_argument = levy_argument
        self.pravy_argument = pravy_argument

    def __eq__(self, other): # Compares operators after they are simplified
        return True if (type(self.simplify()) == type(other.simplify()) and 
                        self.simplify().levy_argument == other.simplify().levy_argument and
                        self.simplify().pravy_argument == other.simplify().pravy_argument) else False


# Base class for elementary functions
class UnaryOperator(Operator):
    def __init__(self, argument):
        self.argument = argument

    def __str__(self):
        return type(self).__name__ + "(" + str(self.argument) + ")"
    
    def __eq__(self, other): # Compares operators after they are simplified
        return True if (type(self.simplify()) == type(other.simplify()) and 
                        self.simplify().argument == other.simplify().argument) else False
    
    def simplify(self):
        self.argument = self.argument.simplify()
        return self


class Addition(BinaryOperator):
    def __str__(self):
        return str(self.levy_argument) + " + " + str(self.pravy_argument)

    def diff(self):
        return self.levy_argument.diff() + self.pravy_argument.diff() # Derivative of a sum: (f + g)' = f' + g'

    def simplify(self):
        # Recursive simplification of operands
        self.levy_argument = self.levy_argument.simplify()
        self.pravy_argument = self.pravy_argument.simplify()
        # Moves a constant always to the left
        if isinstance(self.pravy_argument, cst):
            tmp_arg = self.levy_argument
            self.levy_argument = self.pravy_argument
            self.pravy_argument = tmp_arg
        # Takes a constant out of nested additions: ((cst + a) + b) = (cst + (a + b))
        if isinstance(self.levy_argument, Addition) and isinstance(self.pravy_argument, Addition):
            # ((cst1 + a) + (cst2 + b)) = ((cst1 + cst2) + (a + b))
            if isinstance(self.levy_argument.levy_argument, cst) and isinstance(self.pravy_argument.levy_argument, cst):
                self = (self.levy_argument.levy_argument + self.pravy_argument.levy_argument).simplify() + \
                        (self.levy_argument.pravy_argument + self.pravy_argument.pravy_argument).simplify()
            # ((cst + a) + b) = (cst + (a + b)) # Here b is an addition
            elif isinstance(self.levy_argument.levy_argument, cst):
                self = self.levy_argument.levy_argument + \
                        (self.levy_argument.pravy_argument + self.pravy_argument).simplify()
            # (a + (cst + b)) = (cst + (a + b)) # Here a is an addition
            elif isinstance(self.pravy_argument.levy_argument, cst):
                self = self.pravy_argument.levy_argument + \
                        (self.levy_argument + self.pravy_argument.pravy_argument).simplify()
        if isinstance(self.levy_argument, Addition):
            # ((cst1 + a) + cst2) = ((cst1 + cst2) + a)
            if isinstance(self.levy_argument.levy_argument, cst) and isinstance(self.pravy_argument, cst):
                self = (self.levy_argument.levy_argument + self.pravy_argument).simplify() + self.levy_argument.pravy_argument
            # ((cst + a) + b) = (cst + (a + b))
            elif isinstance(self.levy_argument.levy_argument, cst):
                self = self.levy_argument.levy_argument + \
                        (self.levy_argument.pravy_argument + self.pravy_argument).simplify()
        if isinstance(self.pravy_argument, Addition):
            # (cst1 + (cst2 + b)) = ((cst1 + cst2) + b)
            if isinstance(self.pravy_argument.levy_argument, cst) and isinstance(self.levy_argument, cst):
                self = (self.levy_argument + self.pravy_argument.levy_argument).simplify() + self.pravy_argument.pravy_argument
            # (a + (cst + b)) = (cst + (a + b))
            elif isinstance(self.pravy_argument.levy_argument, cst):
                self = self.pravy_argument.levy_argument + (self.levy_argument + \
                        self.pravy_argument.pravy_argument).simplify()
        # Calculates a sum of two constants immediately
        if isinstance(self.levy_argument, cst) and isinstance(self.pravy_argument, cst):
            return cst(self.levy_argument.value + self.pravy_argument.value)
        # Reduces a sum containing zero operand
        if isinstance(self.levy_argument, cst) and self.levy_argument.value == 0:
            self = self.pravy_argument
        # Shortens a sum of identical operands: a + a = 2a
        elif self.levy_argument == self.pravy_argument:
            self = (2 * self.levy_argument)
        # Shortens a sum of multiples of identical operand: n*a + m*a = (n+m)*a
        elif isinstance(self.levy_argument, Multiplication) and isinstance(self.pravy_argument, Multiplication):
            if self.levy_argument.pravy_argument == self.pravy_argument.pravy_argument:
                self = (((self.levy_argument.levy_argument + self.pravy_argument.levy_argument).simplify() * self.levy_argument.pravy_argument).simplify())
            elif self.levy_argument.pravy_argument == self.pravy_argument.levy_argument:
                self = (((self.levy_argument.levy_argument + self.pravy_argument.pravy_argument).simplify() * self.levy_argument.pravy_argument).simplify())
            elif self.levy_argument.levy_argument == self.pravy_argument.levy_argument:
                self = (((self.levy_argument.pravy_argument + self.pravy_argument.pravy_argument).simplify() * self.pravy_argument.levy_argument).simplify())
            elif self.levy_argument.levy_argument == self.pravy_argument.pravy_argument:
                self = (((self.levy_argument.pravy_argument + self.pravy_argument.levy_argument).simplify() * self.pravy_argument.levy_argument).simplify())
        elif isinstance(self.levy_argument, Multiplication):
            if self.levy_argument.pravy_argument == self.pravy_argument:
                self = (((self.levy_argument.levy_argument + 1).simplify() * self.pravy_argument).simplify())
            elif self.levy_argument.levy_argument == self.pravy_argument:
                self = (((self.levy_argument.pravy_argument + 1).simplify() * self.pravy_argument).simplify())
        elif isinstance(self.pravy_argument, Multiplication):
            if self.levy_argument == self.pravy_argument.pravy_argument:
                self = (((self.pravy_argument.levy_argument + 1).simplify() * self.levy_argument).simplify())
            elif self.levy_argument == self.pravy_argument.levy_argument:
                self = (((self.pravy_argument.pravy_argument + 1).simplify() * self.levy_argument).simplify())
        else:
            pass
        return self


class Multiplication(BinaryOperator):
    def __str__(self):
        # Handling parentheses
        match (self.levy_argument, self.pravy_argument):
            # Left operand is constant
            case l,r if (isinstance(l, cst) and l.value == -1) and \
                        (isinstance(r, UnaryOperator) and not isinstance(r, (mocnina, frc, cst))):
                return "(-" + str(self.pravy_argument) + ")"
            case l,r if (isinstance(l, cst) and l.value == -1) and \
                        (isinstance(r, (mocnina, frc, BinaryOperator))):
                return "(-(" + str(self.pravy_argument) + "))"
            case l,r if (isinstance(l, cst) and l.value < 0) and isinstance(r, identity):
                return "(" + str(self.levy_argument.value) + str(self.pravy_argument) + ")"
            case l,r if (isinstance(l, cst) and l.value >= 0) and isinstance(r, identity):
                return str(self.levy_argument.value) + str(self.pravy_argument)
            # Right operand is constant
            case l,r if (isinstance(r, cst) and r.value == -1) and \
                        (isinstance(l, UnaryOperator) and not isinstance(l, (mocnina, frc, cst))):
                return "(-" + str(self.levy_argument) + ")"
            case l,r if (isinstance(r, cst) and r.value == -1) and \
                        (isinstance(l, (mocnina, frc, BinaryOperator))):
                return "(-(" + str(self.levy_argument) + "))"
            case l,r if (isinstance(r, cst) and r.value < 0) and isinstance(l, identity):
                return "(" + str(self.pravy_argument.value) + str(self.levy_argument) + ")"
            case l,r if (isinstance(r, cst) and r.value >= 0) and isinstance(l, identity):
                return str(self.pravy_argument.value) + str(self.levy_argument)
            # Other cases
            case l,r if (isinstance(l, Multiplication) or (isinstance(l, UnaryOperator) and not isinstance(l, (mocnina, frc)))) and \
                        (isinstance(r, Multiplication) or (isinstance(r, UnaryOperator) and not isinstance(r, (mocnina, frc)))):
                return str(self.levy_argument) + "*" + str(self.pravy_argument)
            case l,r if (isinstance(l, Multiplication) or (isinstance(l, UnaryOperator) and not isinstance(l, (mocnina, frc)))):
                return str(self.levy_argument) + "*(" + str(self.pravy_argument) + ")"
            case l,r if (isinstance(r, Multiplication) or (isinstance(r, UnaryOperator) and not isinstance(r, (mocnina, frc)))):
                return "(" + str(self.levy_argument) + ")*" + str(self.pravy_argument)
            case _:
                return "(" + str(self.levy_argument) + ")*(" + str(self.pravy_argument) + ")"

    def diff(self):
        return self.levy_argument.diff() * self.pravy_argument + self.levy_argument * \
            self.pravy_argument.diff() # Derivative of a product: (f * g)' = f' * g + f * g'
    
    def simplify(self):
        # Recursive simplification of operands
        self.levy_argument = self.levy_argument.simplify()
        self.pravy_argument = self.pravy_argument.simplify()
        # Moves a constant always to the left
        if isinstance(self.pravy_argument, cst):
            tmp_arg = self.levy_argument
            self.levy_argument = self.pravy_argument
            self.pravy_argument = tmp_arg
        # Takes a constant from nested multiplication: ((cst * a) * b) = (cst * (a * b))
        if isinstance(self.levy_argument, Multiplication) and isinstance(self.pravy_argument, Multiplication):
            # ((cst1 * a) * (cst2 * b)) = ((cst1 * cst2) * (a * b))
            if isinstance(self.levy_argument.levy_argument, cst) and isinstance(self.pravy_argument.levy_argument, cst):
                self = (self.levy_argument.levy_argument * self.pravy_argument.levy_argument).simplify() * \
                    (self.levy_argument.pravy_argument * self.pravy_argument.pravy_argument).simplify()
            # ((cst * a) * b) = (cst * (a * b)) # Here b is a multiplication
            elif isinstance(self.levy_argument.levy_argument, cst):
                self = self.levy_argument.levy_argument * \
                    (self.levy_argument.pravy_argument * self.pravy_argument).simplify()
            # (a * (cst * b)) = (cst * (a * b)) # Here a is a multiplication
            elif isinstance(self.pravy_argument.levy_argument, cst):
                self = self.pravy_argument.levy_argument * \
                    (self.levy_argument * self.pravy_argument.pravy_argument).simplify()
        if isinstance(self.levy_argument, Multiplication):
            # ((cst1 * a) * cst2) = ((cst1 * cst2) * a)
            if isinstance(self.levy_argument.levy_argument, cst) and isinstance(self.pravy_argument, cst):
                self = (self.levy_argument.levy_argument * self.pravy_argument).simplify() * \
                    self.levy_argument.pravy_argument
            # ((cst * a) * b) = (cst * (a * b))
            elif isinstance(self.levy_argument.levy_argument, cst):
                self = self.levy_argument.levy_argument * \
                    (self.levy_argument.pravy_argument * self.pravy_argument).simplify()
        if isinstance(self.pravy_argument, Multiplication):
            # (cst1 * (cst2 * b)) = ((cst1 * cst2) * b)
            if isinstance(self.pravy_argument.levy_argument, cst) and isinstance(self.levy_argument, cst):
                self = (self.levy_argument * self.pravy_argument.levy_argument).simplify() * \
                    self.pravy_argument.pravy_argument
            # (a * (cst * b)) = (cst * (a * b))
            elif isinstance(self.pravy_argument.levy_argument, cst):
                self = self.pravy_argument.levy_argument * (self.levy_argument * \
                        self.pravy_argument.pravy_argument).simplify()
        # Calculates a product of two constants immediately
        if isinstance(self.levy_argument, cst) and isinstance(self.pravy_argument, cst):
            self = cst(self.levy_argument.value * self.pravy_argument.value)
        # Reduces products containing unit-constant operands: 1 * a = a
        elif isinstance(self.levy_argument, cst) and self.levy_argument.value == 1:
            self = self.pravy_argument
        # Calculates a multiplication of zero and anything else immediately as zero: 0*a = 0
        elif ((isinstance(self.levy_argument, cst) and self.levy_argument.value == 0) or
              (isinstance(self.pravy_argument, cst) and self.pravy_argument.value == 0)):
            self = cst(0)
        # Shortens a product of identical operands to its power: a*a = a^2
        elif self.levy_argument == self.pravy_argument:
            self = self.levy_argument ** 2
        # Shortens a product of powers of identical operands: (a^n)*(a^m) = a^(n+m)
        elif isinstance(self.levy_argument, mocnina) and self.levy_argument.argument == self.pravy_argument:
            self = self.levy_argument.argument ** (self.levy_argument.exponent + 1)
        elif isinstance(self.pravy_argument, mocnina) and self.pravy_argument.argument == self.levy_argument:
            self = self.pravy_argument.argument ** (self.pravy_argument.exponent + 1)
        elif isinstance(self.levy_argument, mocnina) and isinstance(self.pravy_argument, mocnina) and self.levy_argument.argument == self.pravy_argument.argument:
            self = self.levy_argument.argument ** (self.levy_argument.exponent + self.pravy_argument.exponent)
        else:
            pass
        return self


class cst(UnaryOperator):
    def __init__(self, value):
        self.value = value

    def __str__(self):
        if self.value < 0:
            return "(" + str(self.value) + ")"
        else:
            return str(self.value)
    
    def __eq__(self, other): # Compares operators after they are simplified
        return True if (type(self.simplify()) == type(other.simplify()) and 
                        self.simplify().value == other.simplify().value) else False

    def diff(self):
        return cst(0) # Derivative of a constant: c' = 0

    def simplify(self):
        return self


class identity(UnaryOperator):
    def __init__(self):
        pass

    def __str__(self):
        return "x"

    def __eq__(self, other): # Compares operators after they are simplified
        return True if (type(self.simplify()) == type(other.simplify())) else False
    
    def diff(self):
        return cst(1) # Derivative of an identical function: x' = 1
    
    def simplify(self):
        return self


class mocnina(UnaryOperator):
    def __init__(self, argument, exponent):
        # Supports typing constant arguments as integers or floating-point numbers
        if isinstance(argument, (int, float)):
            self.argument = cst(argument)
        else:
            self.argument = argument
        self.exponent = exponent

    def __str__(self):
        if isinstance(self.argument, (identity, cst)):
            return str(self.argument) + "^" + str(self.exponent)
        else:
            return "(" + str(self.argument) + ")^" + str(self.exponent)

    def __eq__(self, other): # Compares operators after they are simplified
        return True if (type(self.simplify()) == type(other.simplify()) and
                        self.simplify().argument == other.simplify().argument and
                        self.simplify().exponent == other.simplify().exponent) else False

    def diff(self):
        return self.exponent * (self.argument ** (self.exponent - 1)) * \
            self.argument.diff() # Derivative of a power: ((f(x))^n)' = n*((f(x))^(n-1))*(f(x))'

    def simplify(self):
        # Recursive simplification of the operand
        self.argument = self.argument.simplify()
        # Calculating cst^cst
        if isinstance(self.argument, cst) and isinstance(self.exponent, (int, float)):
            return cst(self.argument.value ** self.exponent)
        # Simplification of (f(x))^1 to (f(x))
        if self.exponent == 1:
            self = self.argument
        # Simplification of (f(x))^0 to 1
        elif self.exponent == 0:
            self = cst(1)
        # Simplification of ((f(x))^n)^m to (f(x))^(n*m)
        elif isinstance(self.argument, mocnina):
            self = self.argument.argument ** (self.argument.exponent * self.exponent)
        else:
            pass
        return self


class sin(UnaryOperator):
    def diff(self):
        return cos(self.argument) * self.argument.diff() # ( sin(f(x)) )' = cos(f(x)) * (f(x))'


class cos(UnaryOperator):
    def diff(self):
        return -1 * sin(self.argument) * self.argument.diff() # ( cos(f(x)) )' = -sin(f(x)) * (f(x))'


class exp(UnaryOperator):
    def diff(self):
        return exp(self.argument) * self.argument.diff() # ( e^(f(x)) )' = e^(f(x)) * (f(x))'


class ln(UnaryOperator):
    def diff(self):
        return frc(self.argument) * self.argument.diff() # ( ln(f(x)) )' = ( 1/(f(x)) ) * (f(x))'


class frc(UnaryOperator):
    def __str__(self):
        return "1/(" + str(self.argument) + ")"

    def diff(self):
        return -1 * (frc(self.argument) ** 2) * self.argument.diff() # ( 1/(f(x)) )' = ( -1/((f(x))^2) ) * (f(x))'
    
    def simplify(self):
        # Simplification of 1/(f(x)) to (f(x))^-1
        return ((self.argument ** -1).simplify())

In [109]:
# Tests Bonus implementation
x = identity()
func1 = frc(x**2) * ln(5*x) + cst(5)**3 + (3*x + x)**2 * mocnina(4*x, 3) + 1 + x * -1
func2 = mocnina(x, -2) + (x)
func3 = 2*sin(x + 1) + sin(x + cos(x * x)) + 1 + mocnina(x, 2) + 2 * x + 1 + 2 * mocnina(x, 2)
print(f"                   func1 \t= {func1}")
print(f"        simplified func1 \t= {func1.simplify()}")
print("-----------")
print(f"     derivative of func1 \t= {func1.diff()}")
print(f"       simpl. der. func1 \t= {func1.diff().simplify()}")
print("-----------")
print(f"   der. simplified func1 \t= {func1.simplify().diff()}")
print(f"simpl. der. simpl. func1 \t= {func1.simplify().diff().simplify()}")
print("----------------------------------------------------")
print(f"                   func2 \t= {func2}")
print(f"        simplified func2 \t= {func2.simplify()}")
print("-----------")
print(f"     derivative of func2 \t= {func2.diff()}")
print(f"       simpl. der. func2 \t= {func2.diff().simplify()}")
print("-----------")
print(f"   der. simplified func2 \t= {func2.simplify().diff()}")
print(f"simpl. der. simpl. func2 \t= {func2.simplify().diff().simplify()}")
print("----------------------------------------------------")
print(f"                   func3 \t= {func3}")
print(f"        simplified func3 \t= {func3.simplify()}")
print("-----------")
print(f"     derivative of func3 \t= {func3.diff()}")
print(f"       simpl. der. func3 \t= {func3.diff().simplify()}")
print("-----------")
print(f"   der. simplified func3 \t= {func3.simplify().diff()}")
print(f"simpl. der. simpl. func3 \t= {func3.simplify().diff().simplify()}")
print("----------------------------------------------------")

                   func1 	= (1/(x^2))*ln(5x) + 5^3 + ((3x + x)^2)*((4x)^3) + 1 + (-x)
        simplified func1 	= 126 + (x^-2)*ln(5x) + (4x)^5 + (-x)
-----------
     derivative of func1 	= 0 + (x^-3)*(-2)*1*ln(5x) + (x^-2)*(1/(5x))*(0x + 5*1) + ((4x)^4)*5*(0x + 4*1) + 0x + (-1)*1
       simpl. der. func1 	= (-1) + (-2)*(x^-3)*ln(5x) + 5*(x^-2)*((5x)^-1) + 20*((4x)^4)
-----------
   der. simplified func1 	= 0 + (x^-3)*(-2)*1*ln(5x) + (x^-2)*(1/(5x))*(0x + 5*1) + ((4x)^4)*5*(0x + 4*1) + 0x + (-1)*1
simpl. der. simpl. func1 	= (-1) + (-2)*(x^-3)*ln(5x) + 5*(x^-2)*((5x)^-1) + 20*((4x)^4)
----------------------------------------------------
                   func2 	= x^-2 + x
        simplified func2 	= x^-2 + x
-----------
     derivative of func2 	= (x^-3)*(-2)*1 + 1
       simpl. der. func2 	= 1 + (-2)*(x^-3)
-----------
   der. simplified func2 	= (x^-3)*(-2)*1 + 1
simpl. der. simpl. func2 	= 1 + (-2)*(x^-3)
----------------------------------------------------
                   func3